# AK spruce beetle outbreak risk pipeline

This notebook constitutes the pipeline for producing a dataset of projected climate-driven risk of spruce beetle outbreak for forested areas of Alaska for the 21st century. See the [README](README.md) for more information.

### Outputs

The main product of this pipeline is a 5-D datacube of one categorical variable - climate-driven spruce beetle outbreak risk. The dimensions are:  

* Era (time period)
* Model
* Scenario
* Snowpack level
* Y
* X

##### Format / structure

This will be realized in typical SNAP / ARDAC fashion: a set of GeoTIFFs containing risk values for the entire spatial domain for a single realization of the first four dimension values, i.e. coordinates, and named according to those unique coordinate combinations.

##### Spatial extent

The expected spatial extent of the final dataset is the extent of the forest layer that the final risk data will be masked to. This will come from a version of the binary USFS "Alaska Forest/Non-forest Map" raster (found [here](https://data.fs.usda.gov/geodata/rastergateway/biomass/alaska_forest_nonforest.php)) in SNAP holdings that has been reprojected to EPSG:3338, found at `/workspace/Shared/Tech_Projects/beetles/project_data/ak_forest_mask.tif`.

##### Temporal extent

The risk values will be computed for 30-year long eras of the 21st century:  
* 2010-2039
* 2040-2069
* 2070-2099

### Base data

The base / input data used for computing the climate-driven risk of beetle outbreaks is the "[21st Century Hydrologic Projections for Alaska and Hawaii](https://www.earthsystemgrid.org/dataset/ucar.ral.hydro.predictions.html)" dataset produced by NCAR, specifically the "Alaska Near Surface Meteorology Daily Averages" child dataset. This dataset is available on SNAP infra at `/Data/Base_Data/Climate/AK_NCAR_12km/met`.

## Pipeline steps

0. Setup - Set up path variables, slurm variables, directories, intial conditions, etc.
1. Process yearly risk and risk components
2. Process the final risk class dataset

## 0 - Setup

Set up path variables, slurm variables, directories, intial conditions, etc. Execute this cell before any other step:

In [214]:
import os
from pathlib import Path
import compute_yearly_risk as main
import slurm


ncar_dir = Path(os.getenv("AK_NCAR_DIR"))
base_dir = Path(os.getenv("BASE_DIR"))
output_dir = Path(os.getenv("OUTPUT_DIR"))
scratch_dir = Path(os.getenv("SCRATCH_DIR"))
conda_init_script = Path(os.getenv("CONDA_INIT"))
project_dir = Path(os.getenv("PROJECT_DIR"))
# binary directory from the current conda environment is appended to PATH
path_str = os.getenv("PATH")
# can use this to activate the anaconda-project env
ap_env = Path(path_str.split(":")[0]).parent

# met_dir = Path("/Data/Base_Data/Climate/AK_NCAR_12km/met")
# path to the input meteorological dataset
met_dir = ncar_dir.joinpath("met")

# path to directory where risk components datasets will be written
risk_comp_dir = scratch_dir.joinpath("risk_components")
risk_comp_dir.mkdir(exist_ok=True)

# directory where yearly risk datasets will be written
yearly_risk_dir = scratch_dir.joinpath("yearly_risk")
yearly_risk_dir.mkdir(exist_ok=True)

# directory where risk class dataset will be written
risk_class_dir = scratch_dir.joinpath("risk_class")
risk_class_dir.mkdir(exist_ok=True)

# output direcotry for risk class
out_risk_dir = output_dir.joinpath("risk_class")
out_risk_dir.mkdir(exist_ok=True)

# daymet_comp_fp = scratch_dir.joinpath("yearly_risk_components_daymet.nc")
# path to directory where slurm scripts (jobs and outputs) will be written
slurm_dir = scratch_dir.joinpath("slurm")
slurm_dir.mkdir(exist_ok=True, parents=True)

# script for computing yearly risk for a subset of the data
risk_script = project_dir.joinpath("compute_yearly_risk.py")

# Forest mask of Alaska in EPSG:3338
forest_fp = base_dir.joinpath("ak_forest_mask.tif")


slurm_email = "kmredilla@alaska.edu"
partition = "main"

## 1 - Process yearly risk and risk components

This section creates the yearly risk dataset - a collection of risk values for each year across the grid. This dataset is not expected to be the final product, but it could be a useful intermediate product.

The yearly risk values are calculated from three yearly "risk components". Saving these components as a dataset may have some merit on its own, at least for validation if nothing else. This step utilizes slurm to handle execution of the `compute_yearly_risk.py` script on all model/scenario combinations. 

We will process all future years for the expected final summary time periods: 2008-2099. We will process all years available for the Daymet dataset as well: 1980-2017.

In [18]:
rm /atlas_scratch/kmredilla/beetles/slurm/*

In [19]:
ls /atlas_scratch/kmredilla/beetles/slurm/

In [51]:
# all projections will have years 2010-2099
# need to start with 2008 as yearly risk calculation
#   requires risk components from two years prior
full_future_era = "2008-2099"

In [21]:
import luts


kwargs = {
    "slurm_email": slurm_email,
    "partition": partition,
    "conda_init_script": conda_init_script,
    "ap_env": ap_env,
    "sbatch_fp": sbatch_fp,
    "sbatch_out_fp": sbatch_out_fp,
    "risk_script": risk_script,
    "met_dir": met_dir,
    # template filename for NCAR met data
    "tmp_fn": "{}_{}_BCSD_met_{}.nc4",
}

sbatch_fps = []
for model in luts.models:
    for scenario in luts.scenarios:
        sbatch_fp, sbatch_out_fp = slurm.get_yearly_fps(slurm_dir, model, full_future_era, scenario)
        risk_comp_fp = risk_comp_dir.joinpath(f"risk_components_{model}_{scenario}_{full_future_era}.nc")
        yearly_risk_fp = yearly_risk_dir.joinpath(f"yearly_risk_{model}_{scenario}_{full_future_era}.nc")
        
        kwargs.update({
            "sbatch_fp": sbatch_fp,
            "sbatch_out_fp": sbatch_out_fp,
            "risk_comp_fp": risk_comp_fp,
            "yearly_risk_fp": yearly_risk_fp,
            "era": era,
            "model": model,
            "scenario": scenario,
        })

        slurm.write_sbatch_yearly_risk(**kwargs)
        sbatch_fps.append(sbatch_fp)

We also have the daymet dataset that needs to be processed using different years from all of the projected data. Create an sbatch job for that, too:

In [45]:
model = "daymet"
era = "1980-2017"
sbatch_fp, sbatch_out_fp = slurm.get_yearly_fps(slurm_dir, model, era)
risk_comp_fp = risk_comp_dir.joinpath(f"risk_components_{model}_{era}.nc")
yearly_risk_fp = yearly_risk_dir.joinpath(f"yearly_risk_{model}_{era}.nc")

kwargs.update({
    "sbatch_fp": sbatch_fp,
    "sbatch_out_fp": sbatch_out_fp,
    "met_dir": met_dir,
    "tmp_fn": "daymet_met_{}.nc",
    "risk_comp_fp": risk_comp_fp,
    "yearly_risk_fp": yearly_risk_fp,
    "era": era,
    "model": model,
    "scenario": None,
})

slurm.write_sbatch_yearly_risk(**kwargs)
sbatch_fps.append(sbatch_fp)

Remove existing slurm output files if desired:

In [12]:
# remove existing output files if desired
_ = [fp.unlink() for fp in slurm_dir.glob("*.out")]

Submit the sbatch jobs:

In [22]:
job_ids = [slurm.submit_sbatch(fp) for fp in sbatch_fps]

## 2 - Process the final risk class dataset

Process the yearly risk data into risk classes for the three future eras. Since this doesn't take very long, we can process in the notebook instead of slurming it.



### 2.1 - Prepare files for masking risk class dataset

We want the final risk class dataset to be masked to the forested areas of Alaska, so there is some prep work that needs to happen first:

1. Georeference the NCAR grid and save for a template for regridding the forest mask
2. Re-grid the forest mask (~250m resolution) to match the NCAR template

#### 2.1.1 Georeference NCAR grid

The NCAR data files have only the latitude and longitude geogrids defining the centerpoints of each pixel in the grid - no other spatial information. This is therefore the case for our new risk components and yearly risk datasets. 

To mask our grid to the forested area of Alaska, we want a forest mask raster that is on the same grid as our new datasets, which is the same grid as our expected grid for our risk classes dataset.

So we want to create a GeoTIFF file for the NCAR grid as a template. 

In [159]:
import rioxarray
from wrf import PolarStereographic
from pyproj import Proj, Transformer


# open an NCAR file to get some info from
with xr.open_dataset(met_dir.joinpath("daymet/daymet_met_1980.nc")) as ds:
    # need grid shape below
    ny, nx = ds.longitude.shape
    ncar_arr = np.flipud(ds["tmin"].values[0])

# values provided by NCAR (via email correspondence)
wrf_proj_str = PolarStereographic(**{"TRUELAT1": 64, "STAND_LON": -150}).proj4()
wrf_proj = Proj(wrf_proj_str)
wgs_proj = Proj(proj='latlong', datum='WGS84')
transformer = Transformer.from_proj(wgs_proj, wrf_proj)
e, n = transformer.transform(-150, 64)
# Grid parameters
dx, dy = 12000, 12000
# Down left corner of the domain
x0 = -(nx-1) / 2. * dx + e
y0 = -(ny-1) / 2. * dy + n
# 2d grid
x = np.arange(nx) * dx + x0
y = np.arange(ny) * dy + y0

da = xr.DataArray(
    data=ncar_arr,
    dims=["y", "x"],
    coords=dict(
        y=(["y"], np.flip(y)),
        x=(["x"], x),
    ),
)
da.attrs["_FillValue"] = np.nan

temp_ncar_fp = scratch_dir.joinpath("ncar_template_3338.tif")
da.rio.write_crs(wrf_proj_str).rio.reproject("EPSG:3338").rio.to_raster(temp_ncar_fp)

#### 2.1.2 - Regrid the forest mask

Now regrid the forest mask to match the new NCAR template. 

Since the NCAR data has a larger extent than the forest mask, we will clip (crop) the template file to the extent of the forest mask before regridding the mask.

Create a shapefile to clip to:

In [97]:
import subprocess


cut_fp = scratch_dir.joinpath("clip_ncar.shp")
cut_fp.unlink(missing_ok=True)
_ = subprocess.call(["gdaltindex", cut_fp, forest_fp])

Creating new index file...


Then clip the template:

In [117]:
temp_ncar_clip_fp = scratch_dir.joinpath("ncar_template_clipped_3338.tif")
temp_ncar_clip_fp.unlink(missing_ok=True)
_ = subprocess.call(
    [
        "gdalwarp",
        "-cutline",
        cut_fp,
        "-crop_to_cutline",
        "-q",
        "-overwrite",
        temp_ncar_fp,
        temp_ncar_clip_fp,
    ]
)

Then get the new metadata from the clipped NCAR file:

In [99]:
import rasterio as rio


with rio.open(temp_ncar_clip_fp) as src:
    temp_meta = src.meta

Update the data type and nodata value to match that of existing mask: 

In [100]:
temp_meta.update({"dtype": "uint8", "nodata": 0})

Write a blank array with this metadata to a new GeoTIFF that will serve as a target grid for the original forest mask:

In [150]:
temp_arr = np.zeros((1, temp_meta["height"], temp_meta["width"]), dtype="uint8")

ncar_forest_fp = scratch_dir.joinpath("ak_forest_mask_ncar_3338.tif")
ncar_forest_fp.unlink(missing_ok=True)
with rio.open(ncar_forest_fp, "w", **temp_meta) as src:
    src.write(temp_arr)

Now regrid the original forest mask by calling `gadalwarp` on it with this new target GeoTIFF as the output file. The data of the target file will be updated to match the original file, effectively regridding the original:

In [193]:
_ = subprocess.call(["gdalwarp", "-q", forest_fp, ncar_forest_fp])

### 2.2 - Classify risk and mask

Using our new forest mask and template NCAR file for clipping, we will classify risk, clip, and mask all output GeoTIFFs.

Define the function for classifying risk from the numerical yearly risk values:

In [166]:
def classify_risk(arr):
    """Classify an array of risk values as being either
    low, medium, or high, encoded as 1, 2, or 3, respectively.
    
    Args:
        arr (numpy.ndarray): a 1-D array of risk values
        
    Returns:
        risk_class (int): risk class, either 1, 2, or 3
    """
    # this should only be used on 1-D arrays representing yearly
    #  risk at a single pixel
    assert(len(arr.shape) == 1)
    
    # risk classes are based on whether number of years over some
    #  threshold is greater than half of total years
    half = arr.shape[0] / 2
    
    # high and medium risk thresholds
    high_thr = 0.24
    med_thr = 0.12
    
    if any(np.isnan(arr)):
        risk_class = 0
    elif (arr >= high_thr).sum() >= half:
        risk_class = 3
    elif (arr >= med_thr).sum() >= half:
        risk_class = 2
    else:
        risk_class = 1
        
    return risk_class

Then iterate over the models / scenarios / snow levels / future eras, classify, clip, and mask: 

In [209]:
from itertools import product

args = product(luts.models, luts.scenarios, ["low", "med"], luts.eras)

for model, scenario, snow, era in args:
    yearly_risk_fp = yearly_risk_dir.joinpath(f"yearly_risk_{model}_{scenario}_{full_future_era}.nc")
    start_year, end_year = [int(year) for year in era.split("-")]
    year_sl = slice(start_year, end_year)
    with xr.open_dataset(yearly_risk_fp) as ds:
        era_risk_arr = ds["risk"].sel(year=slice(start_year, end_year), snow=snow).values

    risk_class_arr = np.apply_along_axis(
        classify_risk,
        0,
        ds["risk"].sel(snow=snow, year=year_sl).values,
    ).astype(np.uint8)

    # create an xarray.DataArray for writing risk class
    #  data to georeferenced file
    da = xr.DataArray(
        data=risk_class_arr,
        dims=["y", "x"],
        coords=dict(
            y=(["y"], np.flip(y)),
            x=(["x"], x),
        ),
        attrs={"_FillValue": 0}
    )

    # write to a temporary file for clipping with gdal
    tmp_class_fp = risk_class_dir.joinpath(f"risk_class_{era}_{model}_{scenario}_{snow}.tif")
    da.rio.write_crs(wrf_proj_str).rio.reproject("EPSG:3338").rio.to_raster(tmp_class_fp)
    # run the clip
    class_clip_fp = utils.clip_with_gdal(tmp_class_fp, cut_fp)
    # remove the temporary file
    tmp_class_fp.unlink()
    # Then mask with forest and re-write
    with rio.open(class_clip_fp, "r+") as src:
        with rio.open(ncar_forest_fp) as mask_src:
            arr = src.read(1)
            mask = mask_src.read(1).astype(bool)
            arr[~mask] = 0
            src.write(arr, 1)


Rename these files to get rid of the "_clip" suffix:

In [211]:
_ = [
    fp.rename(fp.parent.joinpath(fp.name.replace("_clip.tif", ".tif")))
    for fp in risk_class_dir.glob("*.tif")
]

And copy these files to `$OUTPUT_DIR` for safe-keeping:

In [216]:
import shutil


copy_args = [(fp, out_risk_dir.joinpath(fp.name)) for fp in risk_class_dir.glob("*.tif")]
_ = [shutil.copy(*arg) for arg in copy_args]

## Pipeline end!

That's it! Beetle risk secured:

In [226]:
print(subprocess.check_output(["ls", out_risk_dir]).decode("utf-8"))

risk_class_2010-2039_CCSM4_rcp45_low.tif
risk_class_2010-2039_CCSM4_rcp45_med.tif
risk_class_2010-2039_CCSM4_rcp85_low.tif
risk_class_2010-2039_CCSM4_rcp85_med.tif
risk_class_2010-2039_GFDL-ESM2M_rcp45_low.tif
risk_class_2010-2039_GFDL-ESM2M_rcp45_med.tif
risk_class_2010-2039_GFDL-ESM2M_rcp85_low.tif
risk_class_2010-2039_GFDL-ESM2M_rcp85_med.tif
risk_class_2010-2039_HadGEM2-ES_rcp45_low.tif
risk_class_2010-2039_HadGEM2-ES_rcp45_med.tif
risk_class_2010-2039_HadGEM2-ES_rcp85_low.tif
risk_class_2010-2039_HadGEM2-ES_rcp85_med.tif
risk_class_2010-2039_MRI-CGCM3_rcp45_low.tif
risk_class_2010-2039_MRI-CGCM3_rcp45_med.tif
risk_class_2010-2039_MRI-CGCM3_rcp85_low.tif
risk_class_2010-2039_MRI-CGCM3_rcp85_med.tif
risk_class_2040-2069_CCSM4_rcp45_low.tif
risk_class_2040-2069_CCSM4_rcp45_med.tif
risk_class_2040-2069_CCSM4_rcp85_low.tif
risk_class_2040-2069_CCSM4_rcp85_med.tif
risk_class_2040-2069_GFDL-ESM2M_rcp45_low.tif
risk_class_2040-2069_GFDL-ESM2M_rcp45_med.tif
risk_class_2040-2069_GFDL-ESM2M_